In [375]:
import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [376]:
import xlrd
wb = xlrd.open_workbook('BikeRentCount.xlsx')
sheets = wb.sheet_names()
MaxRentingRate=[]
RentingRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeRentCount.xlsx', sheet_name=sheets[i])
    rentingRates = data.mean()
    RentingRate.append(rentingRates)
    maxRentingRate = max(rentingRates)
    MaxRentingRate.append(maxRentingRate)
# Example: rate of bike renting at station 2(indexing from 0) during time 8:40-9:00
RentingRate[1][5]

4.717391304347826

In [377]:
# Example: maxmum rate of bike renting at station 2(indexing from 0)
MaxRentingRate[1]

7.923913043478261

In [378]:
wb = xlrd.open_workbook('BikeReturnCount.xlsx')
sheets = wb.sheet_names()
MaxReturnRate=[]
ReturnRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeReturnCount.xlsx', sheet_name=sheets[i])
    returnRates = data.mean()
    ReturnRate.append(returnRates)
    maxReturnRate = max(returnRates)
    MaxReturnRate.append(maxReturnRate)

In [379]:
order_threshold = 2.0
order_up_to = 5.0 
delivery_delay = 20 # in minutes
SIM_RUN = 10 #number of simulation runs
operation_cost = 1 # USD per bike for operation
oil_gas = 2 # USD per 1 refillment
service_fee = 5 # USD per bike per ride
PENALTY = 1 # USD for cost of loss of business oportunity
loss_profit = 0.01 # USD per bike per minute for loss of business opportunity
RunLength = 180

In [380]:
def Rent(**kwargs):
    global revenue, penalty, Loss_profit,happiness
    station_id = kwargs['stationID']
   #NSPP
    interarrival = NSPP(station_id,MaxRentingRate,RentingRate)
    SimFunctions.Schedule(Calendar,"Rent a bike",interarrival,stationID = station_id)
    Loss_profit += loss_profit * Num_bikes[station_id] * interarrival
    if Num_bikes[station_id]>0:
        Num_bikes[station_id]-=1
        revenue += service_fee
    else:
            penalty += PENALTY
            happiness -=1
    if Num_bikes[station_id] <= order_threshold and quantity[station_id] == 0:
        quantity[station_id] = order_up_to - Num_bikes[station_id]
        SimFunctions.Schedule(Calendar,"rebalancing",delivery_delay,stationID=station_id,num_ordered=quantity[station_id])

In [381]:
def Return(**kwargs):
    global  revenue, penalty, Loss_profit,happiness
    station_id = kwargs['stationID']
   #NSPP
    interarrival = NSPP(station_id,MaxReturnRate,ReturnRate)
    SimFunctions.Schedule(Calendar,"Return a bike",interarrival,stationID = station_id)
    Loss_profit += loss_profit * Num_bikes[station_id] * interarrival
    if Num_bikes[station_id] <= Num_docks[station_id]:
        Num_bikes[station_id] += 1
    else:
        happiness-=1

In [382]:
def rebalancing(**kwargs):
    global revenue , cost, Loss_profit,happiness
    station_id = kwargs['stationID']
    num_ordered = kwargs['num_ordered']
    cost += (num_ordered * operation_cost) + oil_gas 
    Num_bikes[station_id] += num_ordered
    quantity[station_id] = 0

In [383]:
def PW_ArrRate(i,t,Rates):
    hour = int(t/20)
    if hour <= 8:
        return Rates[i][hour]
    else:
        return Rates[i][-1]


In [384]:
def NSPP(i,MaxRate,Rate):
    PossibleArrival = SimClasses.Clock + SimRNG.Expon(1/(MaxRate[i]/20), 1)
    while SimRNG.Uniform(0, 1, 1) >= PW_ArrRate(i,PossibleArrival,Rate)/(MaxRate[i]):
        PossibleArrival = PossibleArrival + SimRNG.Expon(1/(MaxRate[i]/20), 1)
    nspp = PossibleArrival - SimClasses.Clock
    return nspp

In [385]:
Calendar = SimClasses.EventCalendar()
TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []
balance_list=[]
happiness_list = []
ZSimRNG = SimRNG.InitializeRNSeed()

#initial number of bikes in each station
Num_bikes=[0,0,0,0,0]

for i in range(SIM_RUN):
    previous_balance=0
    for a in range(5,10):
        for b in range(10,27):
            for c in range(5,11):
                for d in range(5,10):
                    for e in range(5,10):
                        if (a+b+c+d+e)==50:
                            #initial number of bikes in each station in each loop
                            Num_bikes=[a,b,c,d,e]
                            Num_docks=[15,27,11,15,15]
                            #initial number of ordered bikes for rebalancing
                            quantity=[0,0,0,0,0]
                            revenue = 0
                            cost = 0
                            penalty = 0
                            Loss_profit = 0
                            happiness = 0
                            
                            SimClasses.Clock = 0.0
                            SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
                            for i in range(0,5,1):
                                #NSPP
                                SimFunctions.Schedule(Calendar,"Rent a bike",NSPP(i,MaxRentingRate,RentingRate),stationID = i)
                                SimFunctions.Schedule(Calendar,"Return a bike",NSPP(i,MaxReturnRate,ReturnRate),stationID = i) 
                            SimFunctions.Schedule(Calendar,"EndSimulation",RunLength)
                            NextEvent = Calendar.Remove()
                            SimClasses.Clock = NextEvent.EventTime
                            if NextEvent.EventType == 'Rent a bike':
                                Rent(**NextEvent.handlerKwargs)
                            elif NextEvent.EventType == 'Return a bike':
                                Return(**NextEvent.handlerKwargs)
                            elif NextEvent.EventType == 'rebalancing':
                                rebalancing(**NextEvent.handlerKwargs)

                            while NextEvent.EventType != "EndSimulation":
                                NextEvent = Calendar.Remove()
                                SimClasses.Clock = NextEvent.EventTime
                                if NextEvent.EventType == 'Rent a bike':
                                    Rent(**NextEvent.handlerKwargs)
                                elif NextEvent.EventType == 'Return a bike':
                                    Return(**NextEvent.handlerKwargs)
                                elif NextEvent.EventType == 'rebalancing':
                                    rebalancing(**NextEvent.handlerKwargs)
                            #find the best bikes distribition scenario
                            balance = revenue - cost - penalty - Loss_profit
                            if balance>previous_balance:
                                previous_balance=balance
                                bestDistr=[a,b,c,d,e]
    #balance_list.append(balance)
    print(bestDistr)
                            


[7, 22, 9, 5, 7]
[8, 19, 6, 9, 8]
[9, 23, 6, 6, 6]
[7, 24, 5, 8, 6]
[8, 23, 5, 9, 5]
[7, 20, 9, 6, 8]
[6, 25, 8, 5, 6]
[6, 25, 7, 6, 6]
[7, 23, 7, 8, 5]
[6, 24, 6, 7, 7]
